In [20]:
import pandas as pd

In [21]:
#!git clone https://github.com/karen-pal/borges

In [22]:
import os
print(os.getcwd())

c:\GitHub\data_science\Recomendacion


In [23]:
from pathlib import Path
import pandas as pd
import pickle

# Ruta corregida: se usa raw string r'' para evitar errores con '\'
pkls = list(Path(r'./datasets_pkl').glob('*_texts.pkl'))
df = pd.DataFrame()

for pkl in pkls:
    with open(pkl, 'rb') as inp:
        df_ = pickle.load(inp)
        if isinstance(df_, pd.DataFrame):
            if not df_.empty:
                df = pd.concat([df, df_], ignore_index=True)
            else:
                print(f"Este DataFrame está vacío: {pkl}")
        else:
            print(f"¡Ojo! Esto no es un DataFrame: {pkl}")

print("Shape total del DataFrame final:", df.shape)

Shape total del DataFrame final: (719, 3)


In [24]:
df.sample(2)

,link,text_metadata,text
92,https://ciudadseva.com/texto/el-discipulo/,"{'title': 'El discípulo', 'metadata': '[Cuento...","De raso negro, bordeada de armiño y con grueso..."
694,https://ciudadseva.com/texto/la-sombra-vidales/,"{'title': 'La sombra', 'metadata': '[Minicuent...",Yo quería estar solo. Subí las escaleras que c...


In [25]:
# separamos de la metadata el título y autor en sus propias columnas
df['title'] = df['text_metadata'].apply(lambda x: x['title'])
df['author'] = df['text_metadata'].apply(lambda x: x['author'])
df.sample(2)

,link,text_metadata,text,title,author
462,https://ciudadseva.com/texto/una-tarde-plena/,"{'title': 'Una tarde plena', 'metadata': '[Cue...","El saguino¹ es tan pequeño como un ratón, y de...",Una tarde plena,Clarice Lispector
221,https://ciudadseva.com/texto/el-palacio/,"{'title': 'El Palacio', 'metadata': '[Minicuen...","El Palacio no es infinito. Los muros, los terr...",El Palacio,Jorge Luis Borges


In [27]:
# vemos los autores disponibloes
df['author'].value_counts()

author
Jorge Luis Borges             60
Julio Cortázar                55
Baldomero Lillo               50
Juan José Arreola             45
Augusto Monterroso            45
Alfonso Reyes                 37
Enrique Anderson Imbert       36
Mario Benedetti               33
Julio Ramón Ribeyro           27
Roberto Arlt                  25
Clarice Lispector             25
Julio Torri                   23
Felisberto Hernández          15
Luis Vidales                  14
Rubén Darío                   13
Adolfo Bioy Casares           13
Álvaro Mutis                  11
Juan Rodolfo Wilcock          10
Juan Rulfo                    10
Edmundo Valadés               10
Salarrué                       9
Elena Garro                    9
Manuel A. Alonso               9
Eduardo Gudiño Kieffer         8
Alejo Carpentier               8
Juan Bosch                     8
Virgilio Díaz Grullón          7
Andrés Rivera                  7
Silvina Ocampo                 7
Rodolfo Walsh                  6
Ric

In [28]:
# quitamos duplicados y reiniciamos el índice
df = df.drop_duplicates(subset=[c for c in df.columns if c != 'text_metadata'])
df = df.reset_index(drop=True)
df.shape

(693, 5)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pprint import pprint

In [30]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = list(stopwords.words('spanish'))
# eliminamos las "stop words", palabras comunes no informativas
tf = TfidfVectorizer(stop_words=stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\franf\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [35]:
df.head(10)

,link,text_metadata,text,title,author
0,https://ciudadseva.com/texto/1833-1883/,"{'title': '1833-1883', 'metadata': '[Crónica -...",¿Perdemos o ganamos? Cuando hago la pregunta y...,1833-1883,Manuel A. Alonso
1,https://ciudadseva.com/texto/agapito-avellaneda/,"{'title': 'Agapito Avellaneda', 'metadata': '[...",I Conocí hace mucho tiempo a don Jaime Rocafor...,Agapito Avellaneda,Manuel A. Alonso
2,https://ciudadseva.com/texto/el-jibaro-en-la-c...,"{'title': 'El jíbaro en la capital', 'metadata...",Don José de los Reyes Pisafirme es uno de mis ...,El jíbaro en la capital,Manuel A. Alonso
3,https://ciudadseva.com/texto/el-pajaro-malo/,"{'title': 'El pájaro malo', 'metadata': '[Cuen...",Si el lector ha hecho alguna vez el camino de ...,El pájaro malo,Manuel A. Alonso
4,https://ciudadseva.com/texto/el-sueno-de-mi-co...,"{'title': 'El sueño de mi compadre', 'metadata...",Como no podía menos de suceder en la tierra cl...,El sueño de mi compadre,Manuel A. Alonso
5,https://ciudadseva.com/texto/la-gallera/,"{'title': 'La gallera', 'metadata': '[Crónica ...",Puede pasar un pueblo de la isla de Puerto Ric...,La gallera,Manuel A. Alonso
6,https://ciudadseva.com/texto/la-linterna-magica/,"{'title': 'La linterna mágica', 'metadata': '[...","Una de las cosas que distinguen mi carácter, y...",La linterna mágica,Manuel A. Alonso
7,https://ciudadseva.com/texto/la-negrita-y-la-v...,"{'title': 'La negrita y la vaquita', 'metadata...",Era una mañana de cierto mes y año que corría ...,La negrita y la vaquita,Manuel A. Alonso
8,https://ciudadseva.com/texto/perico-paciencia/,"{'title': 'Perico Paciencia', 'metadata': '[Cu...",Tratábase de celebrar la fiesta del santo patr...,Perico Paciencia,Manuel A. Alonso
9,https://ciudadseva.com/texto/alas/,"{'title': 'Alas', 'metadata': '[Minicuento - T...",Yo ejercía entonces la medicina en Humahuaca. ...,Alas,Enrique Anderson Imbert


In [36]:
df.link[0]

'https://ciudadseva.com/texto/1833-1883/'

In [33]:
df.text[0]

'¿Perdemos o ganamos? Cuando hago la pregunta y comparo las dos fechas que encabezan este artículo, acuden a mi mente infinitos recuerdos de hechos que he presenciado en el espacio de cincuenta años y vengo a dar en la sempiterna cuestión que, bajo distinta forma, está planteándose desde tiempo inmemorial y que con relación a Puerto Rico se reduce a averiguar si el progreso realizado entre nosotros nos ha hecho más dichosos, o si con él y por su causa, somos más infelices. Es achaque de viejos, y yo lo soy por mi desgracia, el empeñarse en sostener que en los tiempos de su pasada juventud todo era mejor y más bello. ¿No ha oído el lector más de una vez y entre personas de edad, diálogos parecidos al siguiente? En el toman parte una señora que hoy es abuela, pero en el año cuarenta y dos era una de las señoritas que por su belleza, su juventud y su finura, llamaban más la atención y cautivaban a los elegantes, como entonces se nombraba a los jóvenes en estado de merecer, que ahora llama

In [37]:
# calculamos los features para cada ítem (texto)
tfidf_matrix = tf.fit_transform(df['text'])

In [ ]:
tfidf_matrix.shape

(693, 56608)

In [44]:
# calculamos las similitudes entre todos los documentos
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
n = 5

# diccionario creado para guardar el resultado en un formato (autor - titulo : puntaje, titulo, autor)
results = {} 
for idx, row in df.iterrows():
    # guardamos los indices similares basados en la similitud coseno. Los ordenamos en modo ascendente, siendo 0 nada de similitud y 1 total
    similar_indices = cosine_similarities[idx].argsort()[:-n-2:-1] 
    # guardamos los N más cercanos
    similar_items = [(f"{df['author'][i]} - {df['title'][i]}", round(cosine_similarities[idx][i], 3)) for i in similar_indices]
    results[f"{row['author']} - {row['title']}"] = similar_items[1:]

pprint(results['Jorge Luis Borges - El Aleph'])

[('Jorge Luis Borges - La escritura del dios', np.float64(0.144)),
 ('Jorge Luis Borges - El inmortal', np.float64(0.135)),
 ('Jorge Luis Borges - Utopía de un hombre que está cansado',
  np.float64(0.125)),
 ('Felisberto Hernández - El acomodador', np.float64(0.122)),
 ('Clarice Lispector - La búsqueda de la dignidad', np.float64(0.121))]


In [46]:
print(cosine_similarities)

[[1.         0.1356551  0.18289908 ... 0.04264883 0.06566028 0.04142552]
 [0.1356551  1.         0.12338913 ... 0.01741429 0.05272282 0.05311949]
 [0.18289908 0.12338913 1.         ... 0.02101686 0.06740569 0.05580139]
 ...
 [0.04264883 0.01741429 0.02101686 ... 1.         0.04634811 0.01893521]
 [0.06566028 0.05272282 0.06740569 ... 0.04634811 1.         0.03305628]
 [0.04142552 0.05311949 0.05580139 ... 0.01893521 0.03305628 1.        ]]
